# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902052


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:04<00:58,  2.02s/it]

Rendering models:  10%|▉         | 3/31 [00:06<01:04,  2.30s/it]

Rendering models:  13%|█▎        | 4/31 [00:07<00:44,  1.65s/it]

Rendering models:  16%|█▌        | 5/31 [00:07<00:31,  1.22s/it]

...

Rendering models:  84%|████████▍ | 26/31 [00:10<00:01,  4.55it/s]

Rendering models:  90%|█████████ | 28/31 [00:10<00:00,  5.69it/s]

Rendering models:  94%|█████████▎| 29/31 [00:11<00:00,  4.44it/s]

Rendering models:  97%|█████████▋| 30/31 [00:11<00:00,  4.99it/s]

equidad1                              0.138541
jvillavargas                          0.000584
Thedunkmasta                          0.002580
DrJackie                              0.000815
ElisabethB                            0.007150
Space1000                             0.004532
a.ham.246                             0.000728
Linda_J._Berkel                       0.000492
mattchat                              0.000832
Paddy_van_CH                          0.000654
JEBala                                0.020779
AlphaCentaurii                        0.000843
Mr_DMan                               0.000917
clairedeu                             0.004897
pangeli5                              0.002120
not-logged-in-04ed9ae880f18762cd9e    2.182434
helios1235                            0.000731
Lavadude                              0.023634
not-logged-in-7fd3d99e263d7cff092d    0.002275
barrowwright                          0.000629
ktayl47                               0.000539
Jassels2     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())